In [6]:
import numpy as np

from template_funciones_2 import calcula_L, calcula_lambda


In [ ]:
A = np.array([
   [0, 1, 1, 1, 0, 0, 0, 0],
   [1, 0, 1, 1, 0, 0, 0, 0],
   [1, 1, 0, 1, 0, 1, 0, 0],
   [1, 1, 1, 0, 1, 0, 0, 0],
   [0, 0, 0, 1, 0, 1, 1, 1],
   [0, 0, 1, 0, 1, 0, 1, 1],
   [0, 0, 0, 0, 1, 1, 0, 1],
   [0, 0, 0, 0, 1, 1, 1, 0]
])
L = calcula_L(A)
L

array([[ 3., -1., -1., -1.,  0.,  0.,  0.,  0.],
       [-1.,  3., -1., -1.,  0.,  0.,  0.,  0.],
       [-1., -1.,  4., -1.,  0., -1.,  0.,  0.],
       [-1., -1., -1.,  4., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  4., -1., -1., -1.],
       [ 0.,  0., -1.,  0., -1.,  4., -1., -1.],
       [ 0.,  0.,  0.,  0., -1., -1.,  3., -1.],
       [ 0.,  0.,  0.,  0., -1., -1., -1.,  3.]])

Para encontrar $\Lambda$ tenemos que encontrar el vector $s^{\Lambda}$ optimo que la minimice. Eso equivale a hallar los autovectores de autovalores mas bajos 

In [ ]:
cal